# Python处理excel


使用 [pyexcel](https://pyexcel.readthedocs.io/en/latest/index.html) 处理excel文件，主要使用 按行操作（`get_records`、`save_as`） 和 按列操作（`get_dict`、`save_book_as`）


### 安装 pyexcel

打开cmd，运行`pip install pyexcel pyexcel-xls pyexcel-xlsx` 或者 `pip install -i https://pypi.tuna.tsinghua.edu.cn/simple pyexcel pyexcel-xls pyexcel-xlsx`，如果有提示Permission Denied、权限不足类似的错误信息，则需要在管理员权限打开的cmd中来运行

In [2]:
import pyexcel

假设`student_scores.xlsx`的文件的内容如下：

| name | math_score | english_score |
|:---:|:---:|:---:|
| 张三 | 78 | 85 |
| 李四 | 76 | 87 |
| 王五 | 89 | 91 |

### 第一种方法基于按行操作

`get_records` 返回 excel文件第一个sheet的数据，是以行为单位的*有序*字典列表，第一行每一列作为key（标题），从第二行开始是真正的数据

In [18]:
records = pyexcel.get_records(file_name="student_scores.xlsx")

In [20]:
print(type(records), type(records[0]), records)

<class 'list'> <class 'collections.OrderedDict'> [OrderedDict([('name', '张三'), ('math_score', 78), ('english_score', 85)]), OrderedDict([('name', '李四'), ('math_score', 76), ('english_score', 87)]), OrderedDict([('name', '王五'), ('math_score', 89), ('english_score', 91)])]


*定义三个变量用来存储 数学总成绩、英语总成绩和总人数*，然后使用for循环去遍历每一列的数据，分别针对数学成绩、英语成绩、总人数进行累加，最后求平均值得到最终结果，最后构造一个列表，每一项是一个列表，作为excel每一行的值，然后使用save_as方法保存该数据到excel文件中

In [28]:
math_scores = 0
english_scores = 0
student_count = 0

In [29]:
for record in records:
    math_scores += record['math_score']
    english_scores += record['english_score']
    student_count += 1

In [30]:
average_math_score= math_scores / student_count
average_english_score= english_scores / student_count

In [31]:
print(math_scores, english_scores, student_count, average_math_score, average_english_score)

243 263 3 81.0 87.66666666666667


In [10]:
data = [['average_math_score', 'average_english_score'], [average_math_score, average_english_score]]

In [11]:
pyexcel.save_as(array=data, dest_file_name="summary.xlsx")

### 第二种方法基于按列操作

`get_dict` 同样返回excel文件第一个sheet的数据，不过不是以行为单位的列表，而是以列为单位的*有序*字典，第一行每一列作为key（标题），其值是该列下面所有数据的一个列表

In [21]:
dicts = pyexcel.get_dict(file_name="student_scores.xlsx")

In [23]:
print(type(dicts), type(dicts['name']), dicts)

<class 'collections.OrderedDict'> <class 'pyexcel.internal.sheets.extended_list.PyexcelList'> OrderedDict([('name', ['张三', '李四', '王五']), ('math_score', [78, 76, 89]), ('english_score', [85, 87, 91])])


这里为了计算平均值，使用python里面的`map`、`filter`、`reduce`函数以及`lambda`表达式，可以参考 [Lambda, filter, reduce and map](https://www.python-course.eu/python3_lambda.php)、[Map, Filter and Reduce](http://book.pythontips.com/en/latest/map_filter.html) 

需要注意使用`reduce`的时候需要从`functools`中引入

In [24]:
from functools import reduce

In [38]:
math_scores = reduce((lambda x,y: x + y), dicts['math_score'])
english_scores = reduce((lambda x,y: x + y), dicts['english_score'])
student_count = len(dicts['name'])

In [39]:
average_math_score= math_scores / student_count
average_english_score= english_scores / student_count

In [40]:
print(math_scores, english_scores, student_count, average_math_score, average_english_score)

243 263 3 81.0 87.66666666666667


In [41]:
data = {
    'average_scores': [
        ['average_math_score', 'average_english_score'],
        [average_math_score, average_english_score]
    ]
}

In [42]:
pyexcel.save_book_as(bookdict=data, dest_file_name="summary2.xlsx")